In [ ]:
# 필요한 패키지 설치
!pip install -q --upgrade diffusers transformers accelerate peft

In [ ]:
# 허깅 페이스 로그인
!huggingface-cli login

# Interference with LoRA
- canny controlnet은 U-net에서 Canny Edge 또한
- 이미지 생성을 위한 Denoising의 Condtion으로 활용할 수 있도록 한다.
- 즉, 생성될 이미지의 형태를 Edge로서 제어 가능하다.
- 얼굴의 모양적 특성을 기억하기 위해 사용했다.

In [ ]:
from diffusers import ControlNetModel, AutoencoderKL, StableDiffusionXLControlNetImg2ImgPipeline
from diffusers.utils import load_image, make_image_grid
from PIL import Image
import cv2
import numpy as np
import torch

# orginal_image < 변환을 시작할 이미지
# load_image (이미지 경로나 url)
url = ""
original_image = load_image(url)
image = np.array(original_image)

# 엣지 검출 Threshold 설정
low_threshold = 70
high_threshold = 130

# Canny Edge 생성
image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)
make_image_grid([original_image, canny_image], rows=1, cols=2)

In [ ]:
# canny controlnet 모델 설정
# canny controlnet은 U-net에서 Canny Edge 또한
# 이미지 생성을 위한 Denoising의 Condtion으로 활용할 수 있도록 한다.
# 얼굴의 모양적 특성을 기억하기 위해 사용.
controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True
)
# VAE 설정
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16, use_safetensors=True)
# SDXL Controlnet Pipeline 생성
pipe = StableDiffusionXLControlNetImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

In [ ]:
 # 기본 프로필 모델 LoRA를 불러온다.
 # 사용할 스타일 LoRA의 영향력이 지나치지 않도록 하고
 # 프로필 사진 형태의 이미지가 생성되도록 유도한다.
pipe.load_lora_weights("seoheelee/bpp_lora_small", weight_name="pytorch_lora_weights.safetensors", adapter_name="bpp")

# 원하는 스타일 LoRA를 불러온다.
pipe.load_lora_weights("YOURLORA", weight_name=".safetensors", adapter_name="YOURLORA")

# 각각 반영 비율(adapter_weights), bpp의 비율은 생성될 이미지를 실제 사진에 가깝게 만든다.
# 즉, 실제 사진과 거리가 먼 모델일 수록 비율을 어느정도 줄여야 성능이 좋다.
pipe.set_adapters(["bpp", "YOURLORA"], adapter_weights=[1.0, 1.0])

# 기본 모델의 U-net에 내가 설정한 Adapter를 합친다.
pipe.fuse_lora(adapter_names=["bpp", "YOURLORA"], lora_scale=1.0) # lora scale << 총 반영비율 (0.9나 1.0)

# 합쳤으므로 불러왔던 LoRA weight는 내려준다
pipe.unload_lora_weights()

In [ ]:
# LoRA가 학습에 사용한 instance prompt가 있을시(readme.md에서 확인가능) Trigger로서 prompt에 추가 해줘야 한다.
# 사용된 얼굴 이미지의 성별, 연령대 등 특징을 포함할 시 더 정확한 이미지를 생성한다.
# 이 외에도, 생성될 스타일에 맞게 추가적으로 프롬프트를 설정한다.
# 중요한 것일 수록 앞에 적으며 Theme-Style-Detail 순으로 적어주는 것이 좋다.
# 나타나지 않길 바라는 특징은 부정 프롬프트로 설정한다.
prompt = "YOURTK, a mmpp of USERDATA"
negative_prompt = "colorful, bad quality, ugly, bad anotomy, unrealistic, bad image, noisy"

image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=original_image, # 시작 이미지
    control_image=canny_image, # 시작 이미지의 Canny Image
    controlnet_conditioning_scale=0.5, # Canny Condtioning의 반영 비율
    strength = 0.90, # 시작 이미지의 몇퍼센트를 Noise로 인식하느냐를 의미한다. 즉, 높을수록 원본과 많이 달라질 수 있으며 일반적으로 0.8 이상이 권장된다.
    guidance_scale=10.0 # 프롬프트 반영 비율 (보통 9~15 정도가 적절)
).images[0]
image

In [ ]:
# fuse lora를 통해 기본 모델과 LoRA가 결합되었으므로
# 파라미터들이 최종적으로 결정됐다면 모델 전체를 저장할 수 있다.
# 확장성, 유연성의 문제로 정말 성능이 매우 높지 않는 한 추천하지 않음.
pipe.push_to_hub("")

# IP Adapter
- Style LoRA를 활용할 뿐만 아니라,
- 임의의 이미지를 스타일로서 지정할 수 있다.
- 즉, 제공한 이미지와 같은 스타일 + 내가 제공한 조건으로 이미지를 생성할 수 있다.
- 서비스 제공 환경에서는 다른 유저가 생성한 이미지를 참조할 이미지로 사용하는 것을 고려함.

In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image
import torch

# image to image pipeline을 사용하였다.
# 반영할 이미지의 스타일을 살리기 위해 Canny는 사용하지 않음.
# 필요하다면 Controllnet을 사용한다.
ia_pipeline = AutoPipelineForImage2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")
ia_pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")


# set_ip_adapter_scale은 0~1 사이의 실수로 정할 수도 있지만
# U-net의 블록 별로도 지정하면서 테스트해볼 수 있다. U-net은 Down Sampler와 Up Sampler로 이루어져있다.
# down : down sampler, 이미지 크기를 축소하면서 이미지의 구조적 특징 추출 (layout block)
# up : up sampler, 이미지 크기를 복원하면서 디테일 정보 추가 (style block)
ia_scale = {
    "down": {"block_2": [0.0, 1.0]}, # down sampler의 세번째 블록 두번째 트랜스포머에만 반영
    "up": {"block_0": [0.0, 1.0, 0.0]}, # down sampler의 첫번째 블록 두번째 트랜스포머에만 반영
}
ia_pipeline.set_ip_adapter_scale(ia_scale)

In [ ]:
# LoRA를 이용해 생성할 이미지의 스타일을 추가적으로 지정할 수 있다.
pipe.load_lora_weights("seoheelee/bpp_lora_small", weight_name="pytorch_lora_weights.safetensors", adapter_name="bpp")

# 원하는 스타일 LoRA를 불러온다.
pipe.load_lora_weights("YOURLORA", weight_name=".safetensors", adapter_name="YOURLORA")

# 각각 반영 비율(adapter_weights), bpp의 비율은 생성될 이미지를 실제 사진에 가깝게 만든다.
# 즉, 실제 사진과 거리가 먼 모델일 수록 비율을 어느정도 줄여야 성능이 좋다.
pipe.set_adapters(["bpp", "YOURLORA"], adapter_weights=[1.0, 1.0])

# 기본 모델의 U-net에 내가 설정한 Adapter를 합친다.
pipe.fuse_lora(adapter_names=["bpp", "YOURLORA"], lora_scale=1.0) # lora scale << 총 반영비율 (0.9나 1.0)
ia_pipeline.unload_lora_weights()

In [ ]:
# 스타일로 반영할 이미지
url1 = ""
style_image = load_image(url1)
# 시작 이미지
url2 = ""
original_image = load_image(url2)

# IP Adapter는 영향력이 강하므로, 이를 이용해 이미지를 생성할 때는
# ip adapter scale을 잘 지정해주거나
# 원본 이미지의 정보를 지키기 위해 프롬프트를 구체적으로 설정해주는 것이 좋다.
image = ia_pipeline(
    prompt="YOURTK, a mmpp of USERDATA",
    image=original_image,
    ip_adapter_image=style_image,
    negative_prompt="3d render, monochrom, bad quality, bad anotomy, unrealistic, low quality, worst quality, deformed, glitch, low contrast, noisy",
    strength=0.85,
    guidance_scale=10,
).images[0]
image